In [111]:
import tensorflow as tf
import os
from pathlib import Path

In [112]:
train_path = Path() / 'aclImdb' / 'train'
test_path = Path() / 'aclImdb' / 'test'

In [113]:
unsupervised_path = train_path / 'unsup'
if os.path.exists(unsupervised_path):
    for file in os.listdir(unsupervised_path):
        file_path = unsupervised_path / file
        os.remove(file_path)
    os.rmdir(unsupervised_path)
    print('Unsupervised dataset removed')
else:
    print('Unsupervised dataset does not exist')    

Unsupervised dataset does not exist


In [114]:
train_ds = tf.keras.utils.text_dataset_from_directory(
    train_path,
    batch_size=32,
    seed=42,
)

Found 25000 files belonging to 2 classes.


In [115]:
for i, label in enumerate(train_ds.class_names):
    print("Label", i, "corresponds to", label)

Label 0 corresponds to neg
Label 1 corresponds to pos


In [116]:
(test_ds, val_ds) = tf.keras.utils.text_dataset_from_directory(
    test_path,
    batch_size=32,
    validation_split=0.6,
    subset='both',
    seed=42,
)

Found 25000 files belonging to 2 classes.
Using 10000 files for training.
Using 15000 files for validation.


In [117]:
for i, label in enumerate(test_ds.class_names):
    print("Label", i, "corresponds to", label)

Label 0 corresponds to neg
Label 1 corresponds to pos


In [118]:
vocab_size = 10_000
embedding_dim = 16

In [122]:
text_ds = train_ds.map(lambda x, y: x)

In [124]:
def split_text(text):
    return tf.strings.split(text)

In [125]:
num_words = text_ds.map(split_text).reduce(0, lambda x, y: x + tf.size(y)).numpy()

In [126]:
num_words

5844464

In [119]:
vectorize_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [128]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.GlobalAveragePooling1D(), # this computes the mean embedding for each review :)
    tf.keras.layers.Lambda(lambda x: x * tf.sqrt(tf.cast(num_words, tf.float32))), # multiplies mean embedding the sqrt(num_words)
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [129]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15)

Epoch 1/15
782/782 [==============================] - 17s 21ms/step - loss: 0.4131 - accuracy: 0.8029 - val_loss: 0.3190 - val_accuracy: 0.8672
Epoch 2/15
782/782 [==============================] - 9s 11ms/step - loss: 0.2435 - accuracy: 0.8979 - val_loss: 0.3149 - val_accuracy: 0.8703
Epoch 3/15
782/782 [==============================] - 9s 11ms/step - loss: 0.1899 - accuracy: 0.9231 - val_loss: 0.3605 - val_accuracy: 0.8507
Epoch 4/15
782/782 [==============================] - 9s 11ms/step - loss: 0.1675 - accuracy: 0.9331 - val_loss: 0.3678 - val_accuracy: 0.8693
Epoch 5/15
782/782 [==============================] - 9s 11ms/step - loss: 0.1437 - accuracy: 0.9432 - val_loss: 0.4460 - val_accuracy: 0.8400
Epoch 6/15
782/782 [==============================] - 9s 11ms/step - loss: 0.1250 - accuracy: 0.9512 - val_loss: 0.4961 - val_accuracy: 0.8329
Epoch 7/15
782/782 [==============================] - 9s 11ms/step - loss: 0.1151 - accuracy: 0.9542 - val_loss: 0.6787 - val_accuracy: 0.811